# **Parallel CPU Numerical Computing: Numba ```prange```**

## **Prerequisites**

This tutorial assumes proficiency in Python and the following libraries:

* NumPy
* Numba

Demo System - Benchmarking was performed on a DGX Station A100 320GB

## **Why Numba + ```prange```?**

Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

CPU parallelism is also very accessible.  Numba provides a ```range``` drop-in replacement called ```prange``` that automatically distributes work in for loops to all CPU cores on a local machine.

**When to consider this programming pattern:**
1. You want an easy on-ramp to parallel CPU computing with a performant framework
2. You need higher performance than your single threaded implementation and you do not have GPUs

## **Problem Overview**

In this notebook, We leverage a proxy geospatial nearest neighbor problem to guide us through an evaluation of the Numba's built in CPU multi-processing capability using ```prange```. In this use case, we aim to resolve geospatial observations to their nearest reference points with an added complexity. Our complication adds dynamics to the problem allowing each reference point to move and the set of observations to change on a reoccurring basis. These complexities imply a need to recompute each nearest neighbor at each timestep -- emphasizing the need for high performance techiques. 

Because of its simplicity and arithmetic intensity, we focus our attention on the brute force nearest neighbor technique using the haversine great circle distance formula as our distance metric. This is a popular formula used to calculate the distance between two points on earth.

<center><a href="https://en.wikipedia.org/wiki/Haversine_formula"><img src="./media/haversine-graphic.png" alt="Haversine" style="width: 150;"></a></center></br>

The graphic below illustrates the dynamic nature of our problem. From left to write, we can observe the dynamics of the system at each timestep -- with colored regions representing nearest neighbor decision boundaries for each reference point and points representing observations.

<center><img src="./media/DynamicDecisionBoundaries.png" alt="Visualization" style="width: 1000;"/></center>

In this notebook, we will use all available CPU resources (64 cores, 128 threads) on a problem scaled up by 2048x - 8.8T

**Spoiler Alert -- The GPU techniques each out perform this CPU technique by a long shot.**

Because many of the CPU functions take so long, we use the ```%%time``` magic function and comment out ```%%timeit``` to generate benchmarks.

Since this CPU technique will take a very long time to complete (hours on a DGX Station A100 320GB), were provide an overview of the expected performance measured on a DGX Station A100.

<center><img src="./media/AllScaledCpuGpuPerfTable.png" alt="PerfTable" style="width: 1000;"/></center>

 # **Multi-CPU Experiment**

In [ ]:
from src.solvers import numba_cpu_haversine
from src.simulator import generate_geos
from src.utils import check_accuracy

from numba import jit, prange
import numpy as np
import math

Define constants for the size of our experiment and evaluation criteria.

In [ ]:
N_OBS, N_REF = 2**27, 2**16 # single processor experiment
N_OBS_VAL, N_REF_VAL = 500, 200 # check accuracy
print("Problem Size (N_OBS * N_REF): {:.2f}T".format(N_OBS * N_REF * 1e-12))

## **Numba Multi-CPU Kernel**

Define a JIT function to perform our nearest neighbor calculation. This function adds a ```parallel=True``` argument and leverages the ```prange``` drop-in replacement for ```range``` for an easy on-ramp to performant parallel CPU computing.

In [ ]:
@jit(nopython=True, parallel=True)
def get_nearest(a, b):
    
    out_idx = np.empty(
        (a.shape[0]), dtype=np.uint32)
    
    out_dist = np.empty(
        (a.shape[0]), dtype=np.float32)
    
    for obs_idx in prange(a.shape[0]):
        
        glob_min_dist = 1e11
        glob_min_idx = 0
        
        for ref_idx in range(b.shape[0]):
            
            temp_dist = numba_cpu_haversine(
                a[obs_idx,0],
                a[obs_idx, 1],
                b[ref_idx, 0],
                b[ref_idx, 1])
            
            if temp_dist < glob_min_dist:
                glob_min_dist = temp_dist
                glob_min_idx = ref_idx
        
        out_dist[obs_idx] = glob_min_dist
        out_idx[obs_idx] = glob_min_idx
        
    return out_idx, out_dist

## **Generate Dataset**

Let's generate a scaled up synthetic dataset and validation dataset for our work today using an included utility function. These datasets represent the following:

* ```h_obs``` contains ```N_OBS``` geospatial observations in radians copied to the host from the GPU, used for our full scale benchmark
* ```h_ref``` contains ```N_REF``` geospatial reference points in radians copied to the host from the GPU, used for our full scale benchmark
* ```h_obs_val``` contains ```N_OBS_VAL``` geospatial observations in radians copied to the host from the GPU, used to validate accuracy
* ```h_ref_val``` contains ```N_REF_VAL``` of geospatial reference points in radians copied to the host from the GPU, used to validate accuracy

In [ ]:
h_ref = generate_geos(N_REF, random_state=1).get()
h_obs = generate_geos(N_OBS, random_state=2).get()

h_ref_val = generate_geos(N_REF_VAL, random_state=1).get()
h_obs_val = generate_geos(N_OBS_VAL, random_state=2).get()

## **Validate Accuracy**

Verify our multi-GPU implementation is producing the correct results.

In [ ]:
h_val_idx, h_val_dist = get_nearest(
    h_obs_val, 
    h_ref_val
)

print("Accuracy - Numba CUDA Single GPU:", 
      check_accuracy(
          h_obs_val, 
          h_ref_val,
          h_val_idx, 
          h_val_dist)
     )

## **Benchmark Performance**

We observe our parallel kernel completes in roughly 2hrs 3min 31s on the demo system, hundreds of times slower than the multi-GPU alternatives!

In [ ]:
%%time
#%%timeit
h_val_idx, h_val_dist = get_nearest(
    h_obs, 
    h_ref
)

# **Summarize Results**

In summary, we observe our multi-processing CPU technique solves our full scale problem much faster than the single threaded technique, however, its still orders of magnitude slower than our Multi-GPU techniques.

<img src="./media/MultiScaledCpuGpuPerfTable.png" alt="PerfTable" style="width: 150;"/>

<br>
<div align="left"><h2><b>Please Restart the Kernel<b></h2></div>

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)